In [ ]:
#ptt爬蟲

In [43]:
import requests
import re
def fetch(url):
    
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response

 # step-1

#print(resp.text) # result of setp-1
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries


#print(post_entries)  # result of setp-2
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    domain='https://www.ptt.cc/'
    
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text,
    }

    try:
        # 正常狀況取得資料
        url = urllib.parse.urljoin(domain, ent.find('div.title > a', first=True).attrs['href'])
        resp = requests.get(url)
        content=ent.find('meta.discription', first=True).text
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']
        meta['content']=content
    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    



import urllib.parse

def get_metadata_from(url):

    def parse_next_link(doc):
        html = HTML(html=doc)
        controls = html.find('.action-bar a.btn.wide')
        link = controls[1].attrs.get('href')
        domain='https://www.ptt.cc/'
        return urllib.parse.urljoin(domain, link)

    resp = fetch(url)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    return metadata, next_link

def get_paged_meta(url, num_pages):
    collected_meta = []
    domain='https://www.ptt.cc/'
    for _ in range(num_pages):
        posts, link = get_metadata_from(url)
        collected_meta += posts
        url = urllib.parse.urljoin(domain, link)

    return collected_meta

start_url = 'https://www.ptt.cc/bbs/movie/index.html'
name=start_url.split('/')[-2]
metadata = get_paged_meta(start_url, num_pages=5)
for meta in metadata:
    print( name,meta['title'], meta['date'], meta['author'],meta['content'])


KeyError: 'author'